In [ ]:

SELECT FY,Division, Department, Level1, Nr_Customers, 
CASE WHEN Customer_seq=1 THEN "1stPurchase" WHEN Customer_seq=2 THEN "2ndPurchase"
     WHEN Customer_seq=3 THEN "3rdPurchase" WHEN Customer_seq=4 THEN "4thPurchase"
     WHEN Customer_seq=5 THEN "5thPurchase" WHEN Customer_seq=6 THEN "6thPurchase"
     WHEN Customer_seq=7 THEN "7thPurchase" WHEN Customer_seq=8 THEN "8thPurchase"
     WHEN Customer_seq=9 THEN "9thPurchase" WHEN Customer_seq=10 THEN "10thPurchase"
     WHEN Customer_seq=11 THEN "11thPurchase" WHEN Customer_seq=12 THEN "12thPurchase"
     WHEN Customer_seq=13 THEN "13thPurchase" WHEN Customer_seq=14 THEN "14thPurchase"
     WHEN Customer_seq=15 THEN "15thPurchase" WHEN Customer_seq=16 THEN "16thPurchase"
     WHEN Customer_seq=17 THEN "17thPurchase" WHEN Customer_seq=18 THEN "18thPurchase"
    WHEN Customer_seq=19 THEN "19thPurchase" WHEN Customer_seq=20 THEN "20thPurchase"
ELSE "20th+Purchase" END AS Customer_Seq_Cat,
RepeatPurchase, ROUND(AVG(Days_Till_Next_Order),0) AS Days_Till_Next_Order
FROM (
SELECT FY,Division, Department, Level1, Customer_seq, DATE_DIFF(Auth_Date,Lag,DAY) AS Days_Till_Next_Order,
RepeatPurchase, COUNT(DISTINCT customer_id) AS Nr_Customers
FROM(
SELECT customer_id, FY, Auth_Date,Division, Department, Level1, Customer_seq,
 LAG(Auth_Date) OVER (PARTITION BY customer_id ORDER BY  Auth_Date) AS Lag,
CASE WHEN customer_seq > 1 THEN 'Repeat Customer'
ELSE 'New Customer'
END AS RepeatPurchase
FROM (
    SELECT  *,
         CASE WHEN DATE(x.Auth_Date) >= '2019-04-01' AND DATE(x.Auth_Date) < '2020-04-01' THEN "FY20"
              WHEN DATE(x.Auth_Date) >= '2020-04-01' AND DATE(x.Auth_Date) < '2021-04-01' THEN "FY21"
              WHEN DATE(x.Auth_Date) >= '2021-04-01' AND DATE(x.Auth_Date) < '2022-04-01' THEN "FY22"
      END AS FY,
    RANK() OVER (PARTITION BY customer_id ORDER BY x.Auth_Date) AS customer_seq
    FROM
(
SELECT  
       DISTINCT customer_id,
        bh.bh_division AS Division, 
        bh.bh_reporting_department AS Department,
        bh.bh_level_1 AS Level1,
       DATE(o.order_authed_timestamp) AS Auth_Date,
        o.order_id,
        oi.order_item_sales AS SalesEx,
        oi.order_item_quantity AS Units
    FROM `tal-production-data-bi.tal_dm_order.orders_2010_2020` o,
        UNNEST(o.order_items) oi
    INNER JOIN `tal-production-data-bi.tal_dm_product.business_hierarchy` bh
        ON bh.product_id = oi.product_id
    WHERE   
        DATE(o.order_placed_timestamp) >= "2019-04-01" AND DATE(o.order_placed_timestamp) < "2020-01-01" 
        AND DATE(o.order_authed_timestamp) >= "2019-04-01" AND DATE(o.order_authed_timestamp) < "2020-01-01" 
        AND o.order_authorisation_status IN ('Auth')
        AND o.is_not_return_cancel_in_full IS TRUE
        #AND o.voucher_sales_percent =0
        AND is_test_customer = false
        AND o.order_authed_timestamp IS NOT NULL

UNION ALL

SELECT 
        DISTINCT customer_id,
        bh.bh_division AS Division, 
        bh.bh_reporting_department AS Department,
        bh.bh_level_1 AS Level1,
        DATE(o.order_authed_timestamp) AS Auth_Date,
        o.order_id,
        oi.order_item_sales AS SalesEx,
        oi.order_item_quantity AS Units
    FROM `tal-production-data-bi.tal_dm_order.orders_2020_2030` o,
        UNNEST(o.order_items) oi
    INNER JOIN `tal-production-data-bi.tal_dm_product.business_hierarchy` bh
        ON bh.product_id = oi.product_id
    WHERE   
        DATE(o.order_placed_timestamp) >= "2020-01-01" AND DATE(o.order_placed_timestamp) < "2022-06-01" 
        AND DATE(o.order_authed_timestamp) >= "2020-01-01" AND DATE(o.order_authed_timestamp) < "2022-06-01" 
        AND o.order_authorisation_status IN ('Auth')
        AND o.is_not_return_cancel_in_full IS TRUE
        #AND o.voucher_sales_percent =0
        AND is_test_customer = false
        AND o.order_authed_timestamp IS NOT NULL) x)
WHERE FY IS NOT NULL 
#AND customer_id IN (1743120)
ORDER BY Auth_Date ASC) 
GROUP BY 1,2,3,4,5,6,7
ORDER BY Customer_seq ASC)
GROUP BY 1,2,3,4,5,6,7
ORDER BY RepeatPurchase ASC